In [252]:
#pip install mido

In [253]:
#pip install pygame

In [254]:
#pip install fluidsynth

In [255]:
# you also have to install the fluidynth exe using chocolatey
# see the fluidsynth and chocolatey website for details
# you also have to install the timidity exe from the timidity website

In [256]:
import os
import numpy as np
import mido
import subprocess
import pygame
import random
from mido import Message, MidiFile, MidiTrack

In [257]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

In [258]:
np.array(range(-7,7))

array([-7, -6, -5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5,  6])

In [259]:
# build the music generation algorithm here...

measurelength = 2240

cmajorscale = []
for n in range(72,84): # 0, 12, 24 etc are C. start at 72 to keep the melody in the higher register
    if n%12 in [0,2,4,5,7,9,11]:
        cmajorscale.append(n)

def new_note(current_note):
    # Parameters for the Gaussian probability distribution of the intervals
    mean = 0  # Center of the integers (can adjust)
    std_dev = 2  
    integers = np.array(range(-7,8))
    
    # Calculate the Gaussian probabilities
    probabilities = np.exp(-0.5 * ((integers - mean) / std_dev) ** 2)
    probabilities /= probabilities.sum()  # Normalize to sum to 1
    
    # Generate random indices of the scale vector based on the Gaussian distribution
    interval = np.random.choice(integers, p=probabilities)
    if current_note + interval < 0:
        next_note = current_note + interval + 7
    if current_note + interval > 6:
        next_note = current_note + interval -7
    else: 
        next_note = current_note + interval

    return next_note

cmajchords = np.matrix([[36, 38, 40, 41, 43, 45, 47], # this matrix is used to build diatonic chord progressions in the key of C Maj
[52, 53, 55, 57, 59, 60, 62],
[55, 57, 59, 60, 62, 64, 65],
[59, 60, 62, 64, 65, 67, 69]])

bassnotes = []
i=1
while i < 8: # the number of chords/ bars in the progression
    bassnotes.append(random.randint(0,6))
    i=i+1
print('bassnotes: ', bassnotes)

def build_track(numberofloops):
    l=1
    while l <= numberofloops:
        for i in range(len(bassnotes)):

            # specifications for each chord in the sequence
            for row in range(len(cmajchords)):
                track.append(Message('note_on', note=cmajchords[row,bassnotes[i]], velocity=52, time=0))

            # logic for the melody notes applied to each chord
            melodystart = int(random.choice([measurelength/1,measurelength/2,measurelength/4,0]))
            firstnote = cmajorscale.index(random.choice(cmajorscale))
            track.append(Message('note_on', note=cmajorscale[firstnote], velocity=64, time=melodystart))
            t = melodystart # t is the amount of time since the last chord was played
            r = 1 # r is the number of melody notes since the chord was played
            while t <= measurelength:       
                melodyrhythm = int(random.choice([measurelength/1,measurelength/2,measurelength/4]))
                if r == 1:
                    track.append(Message('note_off', note=cmajorscale[firstnote], velocity=64, time=melodyrhythm))
                    currentnote = firstnote
                    t = t + melodyrhythm
                if r != 1:
                    track.append(Message('note_off', note=cmajorscale[currentnote], velocity=64, time=melodyrhythm)) 
                    currentnote = new_note(currentnote)
                track.append(Message('note_on', note=cmajorscale[currentnote], velocity=64, time=0))
                t = t + melodyrhythm
                r = r + 1
                
            # specifications for cutting off each chord in the sequence
            for row in range(len(cmajchords)):
                track.append(Message('note_off', note=cmajchords[row,bassnotes[i]], velocity=52, time=0)) 
            
        l=l+1

build_track(numberofloops=3) # the number of times the whole progression is played

bassnotes:  [2, 1, 6, 4, 6, 5, 6]


In [260]:
# use fluidsynth, timidity, and pygame to play back the audio out loud
current_folder = os.getcwd()
parent_folder = os.path.dirname(current_folder)
reference_files = parent_folder+'\\reference_files'

mid.save(reference_files+'\\output.mid')
timidity_path = reference_files+'\\TiMidity++-2.15.0\\timidity.exe'
soundfont_path = reference_files+'\\Essential Keys-sforzando-v9.6.sf2'
midi_file_path = reference_files+'\\output.mid'
output_audio_path = reference_files+'\\output.wav'

subprocess.run([timidity_path, '-A', '-Ow', '-o', output_audio_path, '-T', 'wav', '-EFreverb=0', '-EFchorus=0', midi_file_path, soundfont_path])

fluidsynth_path = r'C:\ProgramData\chocolatey\bin\fluidsynth.exe' # TODO: move the file path specifications to an external file
soundfont_path = reference_files+'\\Essential Keys-sforzando-v9.6.sf2'
midi_file_path = reference_files+'\\output.mid'
output_audio_path = reference_files+'\\output.wav'

subprocess.run([fluidsynth_path, '-F', output_audio_path, soundfont_path, midi_file_path])

pygame.init()
pygame.mixer.music.load(reference_files+'\\output.mid')
pygame.mixer.music.play()